# Load Graphs

In [2]:
import os
import glob

path="/home/niklas/Documents/Studium_Uni_Bamberg/Semester4/PWM-PT-HS5/data"

def load_graphs(path):
    path_list=[]
    for folder in os.listdir(path):
        os.chdir(path)
        path_list.append(glob.glob(os.path.join(path,folder,'*.{}'.format("edgelist"))))      
    return path_list



## Load Graph for t=1

In [7]:
import networkx as nx

paths=load_graphs(path)
count=len(paths[0])
number=len(paths)

def create_network(edgelist:str):
    G=nx.read_edgelist(edgelist)
    return G

def create_network_t1(paths:list,count:int,number:int):
    edgelist=paths[number-1][count-1]
    G=create_network(edgelist)
    return G


In [8]:
G=create_network_t1(paths,count,number)
print(G)

Graph with 100 nodes and 1522 edges


## Load Graph for t > 1

In [10]:
def create_network_t(paths:list,t:int,run:int):
    G=None
    for i in range(t):
        edgelist=paths[run][i]
        g=nx.read_edgelist(edgelist)
        if G is None:
            G=g
        else:
            G=nx.compose(G,g)
    return G

In [12]:
G=create_network_t(paths,4,3)
print(G)

Graph with 100 nodes and 3133 edges


# Sparsity/Clustering

In [5]:
nx.average_clustering(G)

0.2972527536619025